# Config


In [117]:
import os
def get_groq_api_key(env_path):
    with open(env_path, "r") as f:
        for line in f:
            if line.startswith("GROQ_API_KEY="):
                return line.strip().split("=", 1)[1]
    raise ValueError("GROQ_API_KEY not found in .env file.")

In [118]:
env_file_path = r"C:\Users\Manideep S\OneDrive - COGNINE\POC-AI-Powered Mental Health Prediction using PHQ-9 (DSM-5)\backend\.env"

names_csv_path = "C:/Users/Manideep S/OneDrive - COGNINE/POC-AI-Powered Mental Health Prediction using PHQ-9 (DSM-5)/tools/misc/random_names.csv"

total_users_required = 1000 #167143

groq_api_key_data_gen = get_groq_api_key(env_file_path)
data_gen_model = "gemma3:1b"

In [119]:
print(groq_api_key_data_gen)

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


In [120]:
import requests
import httpx
import asyncio

# Api


### Register User Synchronous and asynchronous

    emailid: str,
    username: str,
    firstname: str,
    lastname: str,
    age: str,
    gender: str,
    industry: str,
    profession: str,
    password: str,


In [121]:
def register_user(
    emailid: str,
    username: str,
    firstname: str,
    lastname: str,
    age: str,
    gender: str,
    industry: str,
    profession: str,
    password: str,
    api_url: str = "http://localhost:5000/api/register"
):
    payload = {
        "emailid": emailid,
        "username": username,
        "firstname": firstname,
        "lastname": lastname,
        "age": age,
        "gender": gender,
        "industry": industry,
        "profession": profession,
        "password": password,
        "confirm_password": password
    }
    response = requests.post(api_url, json=payload)
    print(response)

In [122]:
register_user("test.user001@gmail.com","test001","firsfdsxxcst","lasxstc","55","male","industry","profession","password")

ConnectionError: HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /api/register (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000235FFF3F140>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [ ]:
async def register_user_async(
    emailid: str,
    username: str,
    firstname: str,
    lastname: str,
    age: str,
    gender: str,
    industry: str,
    profession: str,
    password: str,
    api_url: str = "http://localhost:5000/api/register"
):
    payload = {
        "emailid": emailid,
        "username": username,
        "firstname": firstname,
        "lastname": lastname,
        "age": age,
        "gender": gender,
        "industry": industry,
        "profession": profession,
        "password": password,
        "confirm_password": password
    }
    async with httpx.AsyncClient() as client:
        response = await client.post(api_url, json=payload)
        return response.json()

### Login User Synchronous and asynchronous

    emailid: str,
    password: str,


In [ ]:
def login_user(
    emailid: str,
    password: str,
    api_url: str = "http://localhost:5000/api/login"
):
    payload = {
        "emailid": emailid,
        "password": password
    }
    response = requests.post(api_url, json=payload)
    return response.json()

In [ ]:
async def login_user_async(
    emailid: str,
    password: str,
    api_url: str = "http://localhost:5000/api/login"
):
    payload = {
        "emailid": emailid,
        "password": password
    }
    async with httpx.AsyncClient() as client:
        response = await client.post(api_url, json=payload)
        return response.json()

## PHQ-9 Submission

    user_id: str,
    responses: dict,
    total_score: int,
    doctors_notes: str = "",
    patients_notes: str = "",


### Async PHQ-9 Submission


In [ ]:
async def submit_phq9_form_async(
    user_id: str,
    responses: dict,
    total_score: int,
    doctors_notes: str = "",
    patients_notes: str = "",
    api_url: str = "http://localhost:5000/api/phq9"
):
    payload = {
        "user_id": user_id,
        "responses": responses,
        "totalScore": total_score,
        "doctors_notes": doctors_notes,
        "patients_notes": patients_notes
    }
    async with httpx.AsyncClient() as client:
        response = await client.post(api_url, json=payload)
        return response.json()

### Synchronous PHQ-9 Submission


In [ ]:
def submit_phq9_form(
    user_id: str,
    responses: dict,
    total_score: int,
    doctors_notes: str = "",
    patients_notes: str = "",
    api_url: str = "http://localhost:5000/api/phq9"
):
    payload = {
        "user_id": user_id,
        "responses": responses,
        "totalScore": total_score,
        "doctors_notes": doctors_notes,
        "patients_notes": patients_notes
    }
    response = requests.post(api_url, json=payload)
    return response.json()

## Utils


In [ ]:
import pandas as pd
def read_csv_file(filepath: str):
    return pd.read_csv(filepath)

In [ ]:
import pandas as pd
import numpy as np
import random
import os

class PersistentNamePicker:
    def __init__(self, df, used_csv="used_indices.csv"):
        self.df = df.reset_index(drop=True)
        self.used_csv = used_csv
        self.total_first = len(df["first_name"].unique())
        self.total_last = len(df["last_name"].unique())
        self.all_combinations = self.total_first * self.total_last
        print(f"Total unique first names: {self.total_first}")
        print(f"Total unique last names: {self.total_last}")
        print(f"Total possible unique combinations: {self.all_combinations}")

        # load used combinations if exists
        if os.path.exists(self.used_csv):
            self.used = pd.read_csv(self.used_csv).values.tolist()
        else:
            self.used = []

    def pick(self):
        if len(self.used) >= self.all_combinations:
            raise Exception("All combinations are exhausted!")

        while True:
            fi = random.randint(0, self.total_first - 1)
            li = random.randint(0, self.total_last - 1)
            if [fi, li] not in self.used:
                self.used.append([fi, li])
                pd.DataFrame(self.used, columns=["fi", "li"]).to_csv(self.used_csv, index=False)

                first_name = self.df.loc[fi, "first_name"]
                last_name = self.df.loc[li, "last_name"]
                gender = self.df.loc[fi, "gender"]

                return {
                    "first_name": first_name,
                    "last_name": last_name,
                    "gender": gender,
                    "fullname": f"{first_name} {last_name}"
                }

class NumberOfRecordsPicker:
    def __init__(self, alpha=1.6, beta=1, low=1, high=21):
        self.alpha = alpha
        self.beta = beta
        self.low = low
        self.high = high

    def pick_number(self):
        # sample from Beta distribution
        x = np.random.beta(self.alpha, self.beta)
        # scale to integer range [low, high]
        val = round(self.low + x * (self.high - self.low))
        return min(max(val, self.low), self.high)  # clamp



In [ ]:
picker = NumberOfRecordsPicker()
samples = [picker.pick_number() for _ in range(total_users_required)]
print("Mean:", np.mean(samples))
print("Median:", np.median(samples))


Mean: 13.37
Median: 14.0


In [ ]:
from collections import Counter
counts = Counter(samples)
user_counts = {}
for num in range(1,22):
    print(num,"\t",counts[num])
    user_counts["{num}".format(num=num)] = counts[num]
print(user_counts)


1 	 3
2 	 17
3 	 24
4 	 19
5 	 32
6 	 23
7 	 41
8 	 52
9 	 42
10 	 47
11 	 47
12 	 58
13 	 56
14 	 59
15 	 68
16 	 68
17 	 83
18 	 63
19 	 80
20 	 82
21 	 36
{'1': 3, '2': 17, '3': 24, '4': 19, '5': 32, '6': 23, '7': 41, '8': 52, '9': 42, '10': 47, '11': 47, '12': 58, '13': 56, '14': 59, '15': 68, '16': 68, '17': 83, '18': 63, '19': 80, '20': 82, '21': 36}


# Data Generator


### Register User

    emailid: str,
    username: str,
    firstname: str,
    lastname: str,
    age: str,
    gender: str,
    industry: str,
    profession: str,
    password: str,


In [ ]:
random_names = pd.read_csv(names_csv_path)
random_names = random_names[["first_name","last_name", "gender"]]

In [ ]:
name_picker = PersistentNamePicker(random_names)

Total unique first names: 463
Total unique last names: 361
Total possible unique combinations: 167143


In [ ]:
industry_profession_map = {
    "Information Technology": [
		"Software Engineer",
		"Data Scientist",
		"IT Manager",
		"System Administrator",
		"Web Developer",
		"Mobile Developer",
		"DevOps Engineer",
		"QA Engineer",
		"UI/UX Designer",
		"Product Manager",
		"Business Analyst",
		"Network Engineer",
		"Security Analyst",
		"Database Administrator",
		"Cloud Architect",
		"Technical Support Specialist",
		"IT Consultant",
		"IT Auditor",
		"ERP Specialist",
		"Game Developer",
		"Blockchain Developer",
		"AI/ML Engineer",
		"Other",
	],
	"Healthcare": [
		"Doctor",
		"Nurse",
		"Pharmacist",
		"Therapist",
		"Surgeon",
		"Dentist",
		"Radiologist",
		"Medical Laboratory Technician",
		"Physician Assistant",
		"Paramedic",
		"Healthcare Administrator",
		"Physical Therapist",
		"Occupational Therapist",
		"Psychologist",
		"Dietitian",
		"Veterinarian",
		"Optometrist",
		"Speech Pathologist",
		"Chiropractor",
		"Other",
	],
	"Education": [
		"Teacher",
		"Professor",
		"Lecturer",
		"School Administrator",
		"Counselor",
		"Librarian",
		"Education Consultant",
		"Instructional Designer",
		"Special Education Teacher",
		"Teaching Assistant",
		"Researcher",
		"Coach",
		"Curriculum Developer",
		"Other",
	],
	"Finance": [
		"Accountant",
		"Financial Analyst",
		"Banker",
		"Auditor",
		"Investment Banker",
		"Insurance Agent",
		"Loan Officer",
		"Tax Advisor",
		"Financial Planner",
		"Treasury Analyst",
		"Risk Manager",
		"Compliance Officer",
		"Actuary",
		"Portfolio Manager",
		"Trader",
		"Credit Analyst",
		"Other",
	],
	"Manufacturing": [
		"Engineer",
		"Technician",
		"Supervisor",
		"Operator",
		"Production Manager",
		"Quality Control Inspector",
		"Maintenance Engineer",
		"Process Engineer",
		"Supply Chain Manager",
		"Assembler",
		"Machinist",
		"Safety Officer",
		"Manufacturing Planner",
		"Tool and Die Maker",
		"Other",
	],
	"Retail": [
		"Salesperson",
		"Store Manager",
		"Cashier",
		"Merchandiser",
		"Inventory Manager",
		"Customer Service Representative",
		"Visual Merchandiser",
		"Retail Buyer",
		"Loss Prevention Specialist",
		"E-commerce Specialist",
		"Department Manager",
		"Other",
	],
	"Government": [
		"Civil Servant",
		"Policy Analyst",
		"Administrator",
		"Diplomat",
		"Urban Planner",
		"Public Health Official",
		"Customs Officer",
		"Law Enforcement Officer",
		"Social Worker",
		"Legislative Assistant",
		"Regulatory Affairs Specialist",
		"Other",
	],
	"Construction": [
		"Architect",
		"Civil Engineer",
		"Construction Manager",
		"Site Supervisor",
		"Electrician",
		"Plumber",
		"Carpenter",
		"Surveyor",
		"Safety Manager",
		"Estimator",
		"Project Manager",
		"Mason",
		"Roofer",
		"Other",
	],
	"Hospitality": [
		"Hotel Manager",
		"Chef",
		"Event Planner",
		"Front Desk Agent",
		"Housekeeping Supervisor",
		"Food and Beverage Manager",
		"Travel Agent",
		"Concierge",
		"Restaurant Manager",
		"Bartender",
		"Catering Manager",
		"Other",
	],
	"Transportation & Logistics": [
		"Logistics Coordinator",
		"Supply Chain Analyst",
		"Truck Driver",
		"Warehouse Manager",
		"Dispatcher",
		"Fleet Manager",
		"Customs Broker",
		"Shipping Clerk",
		"Inventory Analyst",
		"Freight Forwarder",
		"Other",
	],
	"Energy & Utilities": [
		"Energy Analyst",
		"Utility Worker",
		"Electrical Engineer",
		"Power Plant Operator",
		"Environmental Engineer",
		"Renewable Energy Specialist",
		"Field Technician",
		"Safety Inspector",
		"Geologist",
		"Other",
	],
	"Legal": [
		"Lawyer",
		"Paralegal",
		"Legal Secretary",
		"Judge",
		"Legal Consultant",
		"Compliance Officer",
		"Court Clerk",
		"Mediator",
		"Legal Analyst",
		"Other",
	],
	"Arts & Entertainment": [
		"Artist",
		"Musician",
		"Actor",
		"Producer",
		"Director",
		"Writer",
		"Graphic Designer",
		"Animator",
		"Photographer",
		"Sound Engineer",
		"Stage Manager",
		"Dancer",
		"Other",
	],
	"Other": ["Other"],
}

In [ ]:
import random

def pick_random_industry_profession():
    industry = random.choice(list(industry_profession_map.keys()))
    profession = random.choice(industry_profession_map[industry])
    return industry, profession

In [ ]:
age=f"{random.randint(18,65)}"
print(type(age))

<class 'str'>


In [ ]:
def create_user_login():
    name = name_picker.pick()

    firstname = name["first_name"]
    lastname = name["last_name"]
    gender = name["gender"]
    username = f"{firstname}.{random.randint(1,9999)}"
    emailid = f"testuser.{username}@gmail.com"
    industry,profession = pick_random_industry_profession()
    age = str(random.randint(18, 65))
    password="Test@1234"

    register_user(emailid=emailid,username=username,firstname=firstname,lastname=lastname,age=age,gender=gender,industry=industry,profession=profession,password=password)
    login_json = login_user(emailid=emailid, password="Test@1234")
    user_id = login_json["user_id"]

    return emailid,user_id,username,firstname,lastname,age,industry,profession,password

    

In [ ]:
def generate_phq9_responses_simple_randomization():
    import random
    return {
            "0": random.randint(0, 3),
            "1": random.randint(0, 3),
            "2": random.randint(0, 3),
            "3": random.randint(0, 3),
            "4": random.randint(0, 3),
            "5": random.randint(0, 3),
            "6": random.randint(0, 3),
            "7": random.randint(0, 3),
            "8": random.randint(0, 3)
            }

In [ ]:
import numpy as np
import random

# Global registry to keep patient trajectories
PATIENT_TRAJECTORIES = {}

# Severity mapping
def severity_label(score: int) -> str:
    if score <= 4:
        return "None"
    elif score <= 9:
        return "Mild"
    elif score <= 14:
        return "Moderate"
    elif score <= 19:
        return "Moderately Severe"
    else:
        return "Severe"

def generate_trajectory(n_total_consults: int, start_score: int) -> list:
    """
    Generate a decreasing trajectory with some noise for PHQ-9 total scores.
    """
    # Base linear decline
    end_score = max(0, int(start_score * 0.3))  # at least 70% improvement possible
    trajectory = np.linspace(start_score, end_score, n_total_consults)

    # Add some noise to simulate fluctuations
    noise = np.random.normal(loc=0, scale=2, size=n_total_consults)  # small ups/downs
    noisy_traj = np.clip(trajectory + noise, 0, 27)  # valid PHQ-9 range

    # Force monotonic decreasing *trend* (but still allow small rises)
    for i in range(1, n_total_consults):
        if noisy_traj[i] > noisy_traj[i-1] + 3:  # prevent huge sudden rise
            noisy_traj[i] = noisy_traj[i-1] + random.choice([0, 1, 2])

    return noisy_traj.round().astype(int).tolist()

def split_into_items(total_score: int) -> dict:
    """
    Split a PHQ-9 total score into 9 random item scores (0–3 each).
    """
    items = [0] * 9
    for _ in range(total_score):
        idx = random.randint(0, 8)
        if items[idx] < 3:
            items[idx] += 1
        else:
            # if full, try again in another slot
            while True:
                idx = random.randint(0, 8)
                if items[idx] < 3:
                    items[idx] += 1
                    break
    return {str(i): items[i] for i in range(9)}

def pick_start_score(n_total_consults: int) -> int:
    """
    Pick a starting severity score biased by number of consultations.
    """
    if n_total_consults <= 3:
        # Mostly low severity, sometimes high
        return np.random.choice(range(0, 20), p=np.linspace(0.1, 0.9, 20)/np.linspace(0.1, 0.9, 20).sum())
    elif n_total_consults <= 10:
        # Spread out, more moderate
        return np.random.choice(range(5, 25), p=np.linspace(0.5, 1.5, 20)/np.linspace(0.5, 1.5, 20).sum())
    else:
        # Long consultations → more severe at start
        return np.random.choice(range(10, 27), p=np.linspace(0.5, 2.0, 17)/np.linspace(0.5, 2.0, 17).sum())

def generate_phq9_response_stateful_randomization(patient_id: str, n_total_consults: int, current_consult: int):
    """
    Generate PHQ-9 responses for a patient at a given consultation.

    Stateful: trajectory is stored for each patient at registration.
    """
    global PATIENT_TRAJECTORIES

    # Register patient if not already present
    if patient_id not in PATIENT_TRAJECTORIES:
        start_score = pick_start_score(n_total_consults)
        trajectory = generate_trajectory(n_total_consults, start_score)
        PATIENT_TRAJECTORIES[patient_id] = trajectory

    # Get current score
    trajectory = PATIENT_TRAJECTORIES[patient_id]
    total_score = trajectory[current_consult - 1]

    # Split into items
    responses = split_into_items(total_score)

    return {
        "patient_id": patient_id,
        "consultation": current_consult,
        "responses": responses,
        "total_score": total_score,
        "severity": severity_label(total_score),
        "n_total_consults": n_total_consults
    }

In [ ]:
# Context prompt for patient notes
patient_prompt = """
    You are the patient writing a private journal entry during your {current_consult} mental health consultation.
    This is your {current_consult} consultation so far.  
    Your latest PHQ-9 responses are: {phq9_responses}, which indicate {severity} depression.

    Write in first-person, describing how you have been feeling, what daily life looks like, 
    any changes since previous consultations, and how symptoms are affecting work, relationships, 
    sleep, appetite, or motivation. Be natural and expressive, like a real person writing down their struggles and reflections.  
    Avoid clinical language — use personal, emotional expression.

    IMPORTANT:
    - output around 40-50 words. Just write the journal entry paragraph
    - do not include any other headings or text like "Journal Entry:", date or "Patient Note:" etc.
    """


# Context prompt for doctor notes
doctor_prompt = """
    You are the doctor reviewing the patient's latest PHQ-9 responses: {phq9_responses}, 
    which indicate {severity} depression.  

    This is the patient’s {current_consult} consultation so far.  
    You also have access to the patient’s last three journal entries: {previous_patient_notes}.  

    Write a professional clinical observation based on DSM-5 criteria and NICE guidelines.  
    Summarize the current symptom profile, any changes or trends compared to previous notes, 
    and highlight risk factors (especially suicidal ideation if present in Q9).  
    Provide an evidence-based interpretation of severity and possible next steps 
    (e.g., therapy, lifestyle interventions, monitoring, referral).  

    IMPORTANT:
    - Write in a structured, professional tone, as if preparing notes for the patient’s record.
    - Output around 40-50 words. Just write the clinical note paragraph
    - do not include any other headings or text like "Clinical Note:", date or "Doctor Note:" etc.
    """


In [ ]:
from ollama import Client

client = Client()

def query_model_ollama(prompt):
    try:
        response = client.chat(model="llama3.1:8b", messages=[{"role": "user", "content": prompt}])
        if "message" in response and "content" in response["message"]:
            return response["message"]["content"].strip()
        else:
            return "Model returned empty output"
    except Exception as e:
        print("Ollama model error:", e)
        return f"Ollama error: {str(e)}"

In [ ]:
import requests

def query_model_groq(prompt, model="gemma3", max_tokens=300, retry=3):
    """
    Query Groq API with a prompt and return model output.
    Optimized for speed and high volume generation.

    prompt: string, prompt to send to model
    model: string, Groq model name (default "gemma-2b-it", fast & free)
    max_tokens: int, max tokens to generate
    retry: int, number of retry attempts on failure
    """

    url = "https://api.groq.com/openai/v1/chat/completions"
    headers = {"Authorization": f"Bearer {groq_api_key_data_gen}"}

    payload = {
        "model": model,
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": max_tokens
    }

    for attempt in range(retry):
        try:
            response = requests.post(url, headers=headers, json=payload, timeout=30)
            resp_json = response.json()

            # Check for choices in response
            if "choices" in resp_json and len(resp_json["choices"]) > 0:
                return resp_json["choices"][0]["message"]["content"].strip()
            
            # If no choices, print error and retry
            print(f"Attempt {attempt+1}: Invalid response, retrying...", resp_json)
        except requests.exceptions.RequestException as e:
            print(f"Attempt {attempt+1}: Request error, retrying...", e)
        except ValueError as e:
            print(f"Attempt {attempt+1}: JSON decode error, retrying...", e)

    # If all retries fail
    return f"API error after {retry} attempts"


In [ ]:
from langchain_groq.chat_models import ChatGroq
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

# Initialize Groq Chat Model with Llama 3.3 70B Versatile
groq_llm = ChatGroq(model="llama-3.3-70b-versatile", api_key=groq_api_key_data_gen)

# Define a chat prompt template
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful and professional assistant."),
    ("user", "{prompt}")
])

# Create a chain to structure the model call
llm_chain = LLMChain(prompt=prompt_template, llm=groq_llm)

def query_model_langchain_groq(prompt: str) -> str:
    try:
        # Generate response from Groq Llama 3.3
        response = llm_chain.run(prompt=prompt)
        return response.strip()
    except Exception as e:
        print("Groq Llama 3.3 error:", e)
        return f"Groq error: {str(e)}"


In [ ]:
import requests

def generate_doctors_patients_notes(phq9_responses, total_score, previous_patient_notes, n_total_consults, current_consult):
    """
    Generates doctor and patient notes based on PHQ-9 responses.

    phq9_responses: dict with keys '0'..'8' and values 0-3
    previous_patient_notes: list of last 3 patient notes (strings)
    n_total_consults: total number of consultations
    current_consult: current consultation number
    groq_api_key_data_gen: API key for Groq
    """

    # Determine PHQ-9 severity
    if total_score <= 4:
        severity = "None-minimal"
    elif total_score <= 9:
        severity = "Mild"
    elif total_score <= 14:
        severity = "Moderate"
    elif total_score <= 19:
        severity = "Moderately Severe"
    else:
        severity = "Severe"

    # Fill prompts
    patient_prompt_filled = patient_prompt.format(
        phq9_responses=phq9_responses,
        severity=severity,
        n_total_consults=n_total_consults,
        current_consult=current_consult
    )

    doctor_prompt_filled = doctor_prompt.format(
        phq9_responses=phq9_responses,
        severity=severity,
        previous_patient_notes=previous_patient_notes,
        n_total_consults=n_total_consults,
        current_consult=current_consult
    )

    # Generate notes
    patient_note = query_model_ollama(patient_prompt_filled)
    doctor_note = query_model_ollama(doctor_prompt_filled)

    return doctor_note, patient_note


In [ ]:
import random

# -----------------------
# Symptom & feeling banks
# -----------------------
symptoms_bank = {
    "None-minimal": [
        "slight fatigue", "occasional low mood", "mild worry", "sometimes feeling distracted",
        "feeling a bit down in the evenings", "minor changes in sleep patterns"
    ],
    "Mild": [
        "feeling tired", "trouble concentrating", "occasional sadness", "low energy during the day",
        "feeling restless", "minor irritability", "difficulty enjoying usual activities"
    ],
    "Moderate": [
        "loss of interest in hobbies", "sleep disturbances", "low motivation", "persistent fatigue",
        "frequent negative thoughts", "difficulty focusing at work", "feeling lonely"
    ],
    "Moderately Severe": [
        "persistent sadness", "difficulty functioning", "frequent irritability", "problems with daily routine",
        "social withdrawal", "significant drop in motivation", "feeling overwhelmed most of the day"
    ],
    "Severe": [
        "hopelessness", "thoughts of self-harm", "inability to get out of bed", "severe anxiety attacks",
        "extreme lack of energy", "loss of interest in eating or personal care",
        "persistent guilt and self-criticism", "intense fear of future"
    ]
}

feelings_bank = {
    "None-minimal": ["okay", "manageable", "slightly low", "a bit tired", "generally fine"],
    "Mild": ["down", "unmotivated", "anxious", "irritable", "uneasy", "restless", "bored"],
    "Moderate": ["sad", "discouraged", "frustrated", "lonely", "overwhelmed", "gloomy", "confused"],
    "Moderately Severe": ["hopeless", "overwhelmed", "exhausted", "disconnected", "distraught", "disheartened"],
    "Severe": ["desperate", "worthless", "completely drained", "trapped", "fearful", "helpless"]
}

daily_impacts = [
    "sleep has been affected", "appetite is irregular", "struggling to concentrate at work",
    "finding it hard to maintain social connections", "energy levels fluctuate during the day",
    "difficulty managing daily chores", "mood swings are noticeable", "motivation is extremely low"
]

doctor_recommendations = {
    "None-minimal": [
        "No active intervention needed. Continue monitoring.",
        "Encourage maintaining healthy routines and self-care."
    ],
    "Mild": [
        "Consider guided self-help or low-intensity CBT.",
        "Recommend mindfulness exercises and monitoring symptoms."
    ],
    "Moderate": [
        "Recommend structured therapy, monitor closely.",
        "Cognitive Behavioral Therapy (CBT) advised, review progress in next visit."
    ],
    "Moderately Severe": [
        "Combination of therapy and possible medication referral.",
        "Suggest referral to mental health specialist, monitor risk factors."
    ],
    "Severe": [
        "Urgent psychiatric evaluation and safety planning required.",
        "Immediate intervention recommended; consider inpatient care if risk escalates."
    ]
}

# -----------------------
# Template banks
# -----------------------
patient_templates = [
    "This is my {current_consult} consultation. Lately, I’ve been {symptom} and {daily_impact}. I feel {feeling}.",
    "During this {current_consult} visit, I noticed that {symptom}. It makes me feel {feeling} and affects my daily life.",
    "On my {current_consult} consultation, I feel {feeling} due to {symptom}. {daily_impact}.",
    "This is consultation number {current_consult}. I have been {symptom} recently, which makes me feel {feeling}. {daily_impact}."
]

doctor_templates = [
    "This is the patient’s {current_consult} consultation. Assessment shows {severity} depression. {recommendation}",
    "Patient's {current_consult} consultation shows {severity} depression. {recommendation}",
    "During consultation {current_consult}, the patient exhibits {severity} depressive symptoms. {recommendation}",
    "Consultation {current_consult}: Patient continues to show {severity} depression. {recommendation}"
]

# -----------------------
# Generator function
# -----------------------
def generate_doctors_patients_notes_templated(phq9_responses, total_score, previous_patient_notes, n_total_consults, current_consult):
    """
    Generates synthetic doctor and patient notes based on PHQ-9 responses using templates.
    previous_patient_notes parameter is kept for compatibility but not used.
    """
    # Determine PHQ-9 severity
    if total_score <= 4:
        severity = "None-minimal"
    elif total_score <= 9:
        severity = "Mild"
    elif total_score <= 14:
        severity = "Moderate"
    elif total_score <= 19:
        severity = "Moderately Severe"
    else:
        severity = "Severe"

    # Randomly pick symptom, feeling, and daily impact
    symptom = random.choice(symptoms_bank[severity])
    feeling = random.choice(feelings_bank[severity])
    daily_impact = random.choice(daily_impacts)
    patient_template = random.choice(patient_templates)
    patient_note = patient_template.format(
        symptom=symptom,
        feeling=feeling,
        daily_impact=daily_impact,
        current_consult=current_consult
    )

    # Prepare doctor note
    recommendation = random.choice(doctor_recommendations[severity])
    doctor_template = random.choice(doctor_templates)
    doctor_note = doctor_template.format(
        severity=severity,
        recommendation=recommendation,
        current_consult=current_consult
    )

    return doctor_note, patient_note


## Iterate through counts


In [124]:
from tqdm import tqdm

for key in user_counts:
    print(key, "\t", user_counts[key])
    
    # Progress bar for users
    for i in tqdm(range(user_counts[key]), desc=f"Users with {key} consultations", unit="user"):
        record_count = int(key)
        emailid, user_id, username, firstname, lastname, age, industry, profession, password = create_user_login()

        last_three_notes = []
        if len(last_three_notes) > 3:
            last_three_notes = last_three_notes[-3:]

        # Progress bar for consultations of each user
        for j in tqdm(range(record_count), desc=f"User {user_id} consultations", leave=False, unit="consult"):
            response = generate_phq9_response_stateful_randomization(
                patient_id=user_id, 
                n_total_consults=record_count, 
                current_consult=j
            )
            
            phq9_response = response["responses"]
            total_score = response["total_score"]
            phq9_severity = response["severity"]
            
            doctors_notes, patients_notes = generate_doctors_patients_notes_templated(
                phq9_responses=phq9_response,
                total_score=total_score,
                previous_patient_notes=last_three_notes,
                n_total_consults=record_count,
                current_consult=j+1
            )
            
            last_three_notes.append({
                "patients_notes": patients_notes,
                "doctors_notes": doctors_notes
            })
            
            submit_phq9_form(
                user_id=user_id, 
                responses=phq9_response, 
                total_score=sum(phq9_response.values()), 
                doctors_notes=doctors_notes, 
                patients_notes=patients_notes
            )


1 	 3


<Response [201]>


<Response [201]>


<Response [201]>





Users with 1 consultations: 100%|██████████| 3/3 [00:20<00:00,  6.71s/user]


...

<Response [201]>


<Response [201]>


<Response [201]>










Users with 8 consultations:   6%|▌         | 3/52 [01:23<22:45, 27.87s/user]


KeyboardInterrupt: 